In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/adware'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-02 19:14:28.926233: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 107 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(107, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(74, 64, 64, 3)

In [6]:
X_test.shape

(33, 64, 64, 3)

In [7]:
y_train.shape

(74, 10)

In [8]:
y_test.shape

(33, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-02 19:14:32.256070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-02 19:14:32.515010: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-02 19:14:32.515030: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-02 19:14:32.515323: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 19:14:32.520245: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-02 19:14:32.5209

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(74, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([9, 1, 7, 4, 6, 6, 8, 3, 6, 4, 5, 6, 1, 9, 4, 3, 4, 5, 7, 4, 2, 1,
       6, 9, 4, 3, 7, 6, 8, 2, 3, 7, 9, 7, 8, 7, 0, 4, 2, 4, 5, 5, 1, 2,
       9, 0, 0, 2, 5, 6, 9, 3, 2, 4, 2, 1, 3, 7, 2, 2, 2, 5, 0, 6, 0, 3,
       9, 3, 0, 4, 8, 0, 1, 5])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-02 19:14:32.698981: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 259ms/step - loss: 0.6141 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4624 - val_loss: 0.3494 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4977
Epoch 2/100
3/3 [==============================] - 0s 45ms/step - loss: 0.3985 - f1_m: 0.0721 - precision_m: 0.2512 - recall_m: 0.0427 - auc: 0.4864 - val_loss: 0.3523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5752
Epoch 3/100
3/3 [==============================] - 0s 44ms/step - loss: 0.3840 - f1_m: 0.0150 - precision_m: 0.0417 - recall_m: 0.0091 - auc: 0.5139 - val_loss: 0.3191 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.6187
Epoch 4/100
3/3 [==============================] - 0s 49ms/step - loss: 0.3729 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5295 - val_loss: 0.3333 - val_f1_

3/3 [==============================] - 0s 44ms/step - loss: 0.2040 - f1_m: 0.4767 - precision_m: 0.8681 - recall_m: 0.3286 - auc: 0.9318 - val_loss: 0.2736 - val_f1_m: 0.1026 - val_precision_m: 0.2857 - val_recall_m: 0.0625 - val_auc: 0.7908
Epoch 32/100
3/3 [==============================] - 0s 44ms/step - loss: 0.2014 - f1_m: 0.4172 - precision_m: 0.7324 - recall_m: 0.2951 - auc: 0.9228 - val_loss: 0.2705 - val_f1_m: 0.1026 - val_precision_m: 0.2857 - val_recall_m: 0.0625 - val_auc: 0.7939
Epoch 33/100
3/3 [==============================] - 0s 43ms/step - loss: 0.1789 - f1_m: 0.5196 - precision_m: 0.7886 - recall_m: 0.3898 - auc: 0.9488 - val_loss: 0.2731 - val_f1_m: 0.1538 - val_precision_m: 0.4286 - val_recall_m: 0.0938 - val_auc: 0.7854
Epoch 34/100
3/3 [==============================] - 0s 43ms/step - loss: 0.1902 - f1_m: 0.5497 - precision_m: 0.8256 - recall_m: 0.4154 - auc: 0.9382 - val_loss: 0.2854 - val_f1_m: 0.1316 - val_precision_m: 0.4167 - val_recall_m: 0.0781 - val_auc: 

3/3 [==============================] - 0s 44ms/step - loss: 0.0636 - f1_m: 0.8956 - precision_m: 0.9425 - recall_m: 0.8531 - auc: 0.9957 - val_loss: 0.3395 - val_f1_m: 0.1964 - val_precision_m: 0.2292 - val_recall_m: 0.1719 - val_auc: 0.7962
Epoch 64/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0678 - f1_m: 0.8218 - precision_m: 0.8516 - recall_m: 0.7948 - auc: 0.9924 - val_loss: 0.3251 - val_f1_m: 0.1864 - val_precision_m: 0.2037 - val_recall_m: 0.1719 - val_auc: 0.7915
Epoch 65/100
3/3 [==============================] - 0s 45ms/step - loss: 0.0579 - f1_m: 0.9074 - precision_m: 0.9194 - recall_m: 0.8964 - auc: 0.9943 - val_loss: 0.3093 - val_f1_m: 0.6754 - val_precision_m: 0.7000 - val_recall_m: 0.6562 - val_auc: 0.7947
Epoch 66/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0678 - f1_m: 0.8697 - precision_m: 0.9078 - recall_m: 0.8352 - auc: 0.9958 - val_loss: 0.3000 - val_f1_m: 0.6786 - val_precision_m: 0.7083 - val_recall_m: 0.6562 - val_auc: 

3/3 [==============================] - 0s 44ms/step - loss: 0.0402 - f1_m: 0.9388 - precision_m: 0.9388 - recall_m: 0.9388 - auc: 0.9974 - val_loss: 0.3797 - val_f1_m: 0.1967 - val_precision_m: 0.2069 - val_recall_m: 0.1875 - val_auc: 0.8035
Epoch 96/100
3/3 [==============================] - 0s 46ms/step - loss: 0.0318 - f1_m: 0.8810 - precision_m: 0.8810 - recall_m: 0.8810 - auc: 0.9982 - val_loss: 0.3790 - val_f1_m: 0.2000 - val_precision_m: 0.2143 - val_recall_m: 0.1875 - val_auc: 0.7996
Epoch 97/100
3/3 [==============================] - 0s 44ms/step - loss: 0.0447 - f1_m: 0.9219 - precision_m: 0.9219 - recall_m: 0.9219 - auc: 0.9975 - val_loss: 0.3311 - val_f1_m: 0.2167 - val_precision_m: 0.2321 - val_recall_m: 0.2031 - val_auc: 0.8163
Epoch 98/100
3/3 [==============================] - 0s 43ms/step - loss: 0.0454 - f1_m: 0.8909 - precision_m: 0.8949 - recall_m: 0.8870 - auc: 0.9961 - val_loss: 0.3006 - val_f1_m: 0.2143 - val_precision_m: 0.2500 - val_recall_m: 0.1875 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3089 - f1_m: 0.7069 - precision_m: 0.7308 - recall_m: 0.6875 - auc: 0.7959


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3089 - f1_m: 0.7069 - precision_m: 0.7308 - recall_m: 0.6875 - auc: 0.7959


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.7068964242935181
